In [ ]:
# !pip install -qU  langchain-core
# !pip install -qU langchain-community
# !pip install -qU  "crewai[tools]"
# !pip install -qU langchain-groq

In [1]:
import json
import os
import sqlite3
from dataclasses import asdict, dataclass
from datetime import datetime, timezone
from pathlib import Path
from textwrap import dedent
from typing import Any, Dict, List, Tuple, Union

import pandas as pd
from crewai import Agent, Crew, Process, Task
from crewai_tools import tool

# from google.colab import userdata
from langchain.schema import AgentFinish
from langchain.schema.output import LLMResult
from langchain_community.tools.sql_database.tool import (
    InfoSQLDatabaseTool,
    ListSQLDatabaseTool,
    QuerySQLCheckerTool,
    QuerySQLDataBaseTool,
)
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_core.callbacks.base import BaseCallbackHandler
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

# from google.colab import userdata

from dotenv import load_dotenv

load_dotenv()

os.environ["GROQ_API_KEY"] = os.environ.get("GROQ_API_KEY")
os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY")

In [2]:
df = pd.read_csv("ds-salaries.csv")
df.head()

,Unnamed: 0,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L


In [3]:
try:
    os.remove("salaries.db")
except:
    pass

connection = sqlite3.connect("salaries.db")
df.to_sql(name="salaries", con=connection)

607

In [4]:
@dataclass
class Event:
    event: str
    timestamp: str
    text: str


def _current_time() -> str:
    return datetime.now(timezone.utc).isoformat()


class LLMCallbackHandler(BaseCallbackHandler):
    def __init__(self, log_path: Path):
        self.log_path = log_path

    def on_llm_start(
        self, serialized: Dict[str, Any], prompts: List[str], **kwargs: Any
    ) -> Any:
        """Run when LLM starts running."""
        assert len(prompts) == 1
        event = Event(event="llm_start", timestamp=_current_time(), text=prompts[0])
        with self.log_path.open("a", encoding="utf-8") as file:
            file.write(json.dumps(asdict(event)) + "\n")

    def on_llm_end(self, response: LLMResult, **kwargs: Any) -> Any:
        """Run when LLM ends running."""
        generation = response.generations[-1][-1].message.content
        event = Event(event="llm_end", timestamp=_current_time(), text=generation)
        with self.log_path.open("a", encoding="utf-8") as file:
            file.write(json.dumps(asdict(event)) + "\n")

In [5]:
llm = ChatGroq(
    temperature=0,
    model_name="llama3-70b-8192",
    # model_name="mixtral-8x7b-32768",
    callbacks=[LLMCallbackHandler(Path("prompts.jsonl"))],
)

In [6]:
db = SQLDatabase.from_uri("sqlite:///salaries.db")

In [7]:
@tool("list_tables")
def list_tables() -> str:
    """List the available tables in the database"""
    return ListSQLDatabaseTool(db=db).invoke("")


list_tables.run()

Using Tool: list_tables


'salaries'

In [8]:
@tool("tables_schema")
def tables_schema(tables: str) -> str:
    """
    Input is a comma-separated list of tables, output is the schema and sample rows
    for those tables. Be sure that the tables actually exist by calling `list_tables` first!
    Example Input: table1, table2, table3
    """
    tool = InfoSQLDatabaseTool(db=db)
    return tool.invoke(tables)


print(tables_schema.run("salaries"))

Using Tool: tables_schema

CREATE TABLE salaries (
	"index" INTEGER, 
	"Unnamed: 0" INTEGER, 
	work_year INTEGER, 
	experience_level TEXT, 
	employment_type TEXT, 
	job_title TEXT, 
	salary INTEGER, 
	salary_currency TEXT, 
	salary_in_usd INTEGER, 
	employee_residence TEXT, 
	remote_ratio INTEGER, 
	company_location TEXT, 
	company_size TEXT
)

/*
3 rows from salaries table:
index	Unnamed: 0	work_year	experience_level	employment_type	job_title	salary	salary_currency	salary_in_usd	employee_residence	remote_ratio	company_location	company_size
0	0	2020	MI	FT	Data Scientist	70000	EUR	79833	DE	0	DE	L
1	1	2020	SE	FT	Machine Learning Scientist	260000	USD	260000	JP	0	JP	S
2	2	2020	SE	FT	Big Data Engineer	85000	GBP	109024	GB	50	GB	M
*/


In [9]:
@tool("execute_sql")
def execute_sql(sql_query: str) -> str:
    """Execute a SQL query against the database. Returns the result"""
    return QuerySQLDataBaseTool(db=db).invoke(sql_query)


execute_sql.run("SELECT * FROM salaries WHERE salary > 10000 LIMIT 5")

Using Tool: execute_sql


"[(0, 0, 2020, 'MI', 'FT', 'Data Scientist', 70000, 'EUR', 79833, 'DE', 0, 'DE', 'L'), (1, 1, 2020, 'SE', 'FT', 'Machine Learning Scientist', 260000, 'USD', 260000, 'JP', 0, 'JP', 'S'), (2, 2, 2020, 'SE', 'FT', 'Big Data Engineer', 85000, 'GBP', 109024, 'GB', 50, 'GB', 'M'), (3, 3, 2020, 'MI', 'FT', 'Product Data Analyst', 20000, 'USD', 20000, 'HN', 0, 'HN', 'S'), (4, 4, 2020, 'SE', 'FT', 'Machine Learning Engineer', 150000, 'USD', 150000, 'US', 50, 'US', 'L')]"

In [10]:
@tool("check_sql")
def check_sql(sql_query: str) -> str:
    """
    Use this tool to double check if your query is correct before executing it. Always use this
    tool before executing a query with `execute_sql`.
    """
    return QuerySQLCheckerTool(db=db, llm=llm).invoke({"query": sql_query})


check_sql.run("SELECT * WHERE salary > 10000 LIMIT 5 table = salaries")

Using Tool: check_sql


'SELECT * FROM salaries WHERE salary > 10000 LIMIT 5'

In [11]:
sql_dev = Agent(
    role="Senior Database Developer",
    goal="Construct and execute SQL queries based on a request",
    backstory=dedent(
        """
        You are an experienced database engineer who is master at creating efficient and complex SQL queries.
        You have a deep understanding of how different databases work and how to optimize queries.
        Use the `list_tables` to find available tables.
        Use the `tables_schema` to understand the metadata for the tables.
        Use the `execute_sql` to check your queries for correctness.
        Use the `check_sql` to execute queries against the database.
    """
    ),
    # llm=llm,
    tools=[list_tables, tables_schema, execute_sql, check_sql],
    allow_delegation=False,
)

In [12]:
data_analyst = Agent(
    role="Senior Data Analyst",
    goal="You receive data from the database developer and analyze it",
    backstory=dedent(
        """
        You have deep experience with analyzing datasets using Python.
        Your work is always based on the provided data and is clear,
        easy-to-understand and to the point. You have attention
        to detail and always produce very detailed work (as long as you need).
    """
    ),
    # llm=llm,
    allow_delegation=False,
)

In [13]:
report_writer = Agent(
    role="Senior Report Editor",
    goal="Write an executive summary type of report based on the work of the analyst",
    backstory=dedent(
        """
        Your writing still is well known for clear and effective communication.
        You always summarize long texts into bullet points that contain the most
        important details.
        """
    ),
    # llm=llm,
    allow_delegation=False,
)

In [14]:
extract_data = Task(
    description="Extract data that is required for the query {query}.",
    expected_output="Database result for the query",
    agent=sql_dev,
)

In [15]:
analyze_data = Task(
    description="Analyze the data from the database and write an analysis for {query}.",
    expected_output="Detailed analysis text",
    agent=data_analyst,
    context=[extract_data],
)

In [17]:
write_report = Task(
    description=dedent(
        """
        Write an executive summary of the report from the analysis. The report
        must be less than 100 words.
    """
    ),
    expected_output="Markdown report",
    agent=report_writer,
    context=[analyze_data],
)

In [18]:
crew = Crew(
    agents=[sql_dev, data_analyst, report_writer],
    tasks=[extract_data, analyze_data, write_report],
    process=Process.sequential,
    verbose=True,
    memory=False,
    output_log_file="crew.log",
)

In [ ]:
# Kickoff The Crew For Query 1:

inputs = {
    "query": "Effects on salary (in USD) based on company location, size and employee experience"
}

result = crew.kickoff(inputs=inputs)

# Agent: Senior Database Developer
## Task: Extract data that is required for the query Effects on salary (in USD) based on company location, size and employee experience.


# Agent: Senior Database Developer
## Thought: I need to determine the available tables in the database that might contain relevant information regarding salary, company location, size, and employee experience.
## Using tool: list_tables
## Tool Input: 
"{}"
## Tool Output: 
salaries


# Agent: Senior Database Developer
## Thought: Thought: The only available table is 'salaries'. I will now check the schema of this table to understand its structure and see if it contains the necessary information for the query.
## Using tool: tables_schema
## Tool Input: 
"{\"tables\": \"salaries\"}"
## Tool Output: 

CREATE TABLE salaries (
	"index" INTEGER, 
	"Unnamed: 0" INTEGER, 
	work_year INTEGER, 
	experience_level TEXT, 
	employment_type TEXT, 
	job_title TEXT, 
	salary INTEGER, 
	salary_currency TEXT, 
	salary_in_usd INTEG

In [20]:
# Kickoff The Crew For Query 2:

inputs = {
    "query": "How is the `Machine Learning Engineer` salary in USD is affected by remote positions"
}

result = crew.kickoff(inputs=inputs)

# Agent: Senior Database Developer
## Task: Extract data that is required for the query How is the `Machine Learning Engineer` salary in USD is affected by remote positions.


# Agent: Senior Database Developer
## Thought: I need to first determine the structure of the database to identify the relevant tables that might contain salary data for Machine Learning Engineers and their correlation with remote positions.
## Using tool: list_tables
## Tool Input: 
"{}"
## Tool Output: 
salaries


# Agent: Senior Database Developer
## Thought: Thought: I have found a table called "salaries". Next, I will check the schema of the "salaries" table to understand its structure and determine how to retrieve the necessary information regarding Machine Learning Engineer salaries and their relation to remote positions.
## Using tool: tables_schema
## Tool Input: 
"{\"tables\": \"salaries\"}"
## Tool Output: 

CREATE TABLE salaries (
	"index" INTEGER, 
	"Unnamed: 0" INTEGER, 
	work_year INTEGER, 
	experi

In [ ]:
# Kickoff The Crew For Query 3:

inputs = {
    "query": "How is the salary in USD based on employment type and experience level? "
}

result = crew.kickoff(inputs=inputs)